In [2]:
#Import necessary/useful packages
import kwant
import numpy as np
import tinyarray as tiny
from types import SimpleNamespace
import scipy.sparse.linalg as la
from matplotlib import pyplot as plt
import holoviews as hv
hv.extension('matplotlib', logo=False) #BOKEH DOESNT SUPPORT LATEX IN LABELS
from holoviews import opts
hv.output(fig='svg')

# define the Pauli matrices
s_0 = np.identity(2)
s_z = np.array([[1, 0], [0, -1]])
s_x = np.array([[0, 1], [1, 0]])
s_y = np.array([[0, -1j], [1j, 0]])
# pauli = [sx,sy,sz] #pauli = {'x':sx, 'y':sy, 'z':sz} #as dictionary

#define the Pauli matrices 
pauli = SimpleNamespace(s0=np.array([[1., 0.], [0., 1.]]),
                        sx=np.array([[0., 1.], [1., 0.]]),
                        sy=np.array([[0., -1j], [1j, 0.]]),
                        sz=np.array([[1., 0.], [0., -1.]]))

#extend Pauli matrices to particle-hole space (see e.g. BdG 'trick' paper)
pauli.s0s0 = np.kron(pauli.s0, pauli.s0) #I(4)
pauli.s0sx = np.kron(pauli.s0, pauli.sx) # 4x4 \sigma_x
pauli.s0sy = np.kron(pauli.s0, pauli.sy) # 4x4 \sigma_y
pauli.s0sz = np.kron(pauli.s0, pauli.sz) # 4x4 \sigma_z
pauli.sxs0 = np.kron(pauli.sx, pauli.s0) 
pauli.sxsx = np.kron(pauli.sx, pauli.sx)
pauli.sxsy = np.kron(pauli.sx, pauli.sy)
pauli.sxsz = np.kron(pauli.sx, pauli.sz)
pauli.sys0 = np.kron(pauli.sy, pauli.s0)
pauli.sysx = np.kron(pauli.sy, pauli.sx)
pauli.sysy = np.kron(pauli.sy, pauli.sy) # \tau_x for superconducting terms ?
pauli.sysz = np.kron(pauli.sy, pauli.sz)
pauli.szs0 = np.kron(pauli.sz, pauli.s0) # \tau_z for chemical potential
pauli.szsx = np.kron(pauli.sz, pauli.sx) # \sigma_x ?
pauli.szsy = np.kron(pauli.sz, pauli.sy)
pauli.szsz = np.kron(pauli.sz, pauli.sz)

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}$$
In the Nambu spinor basis
\begin{equation}
\psi^{\dagger}(\mathbf{r}) = \left( c^{\dagger}_{\uparrow}(\mathbf{r}) c^{\dagger}_{\downarrow}(\mathbf{r}) \ 
    c_{\uparrow}(\mathbf{r}) -c_{\downarrow}(\mathbf{r}) \right)
\end{equation}
the Hamiltonian from the skyrmion paper <a href='https://arxiv.org/abs/1904.03005'>[1]</a> reads
\begin{equation}
H(\mathbf{r}) =   \left(  - \frac{1}{2m}\nabla^2 - \mu \right)\tau_z + J \sigma_{\mathbf{r}} \cdot \mathbf{B(r)} + \Delta_0 \tau_x.
\end{equation}
Let's focus only on the magnetic field for now and see if we can get a nice spectrum.
\begin{equation} \tau_z = \sigma_z \otimes I_2 = \begin{pmatrix} I_2 & 0 \\ 0 & -I_2 \end{pmatrix} \to \mu c^{\dagger}_{\uparrow}c_{\uparrow} \end{equation}
\begin{equation} \tau_x = \sigma_x \otimes I_2 =\begin{pmatrix} 0 & I_2 \\ I_2 & 0 \end{pmatrix} \to \Delta c^{\dagger}_{\uparrow}c^{\dagger}_{\uparrow} \end{equation}
\begin{equation} \sigma_z = I_2 \otimes \sigma_z = \begin{pmatrix} \sigma_z & 0 \\ 0 & \sigma_z \end{pmatrix} \to B_z(c^{\dagger}_{\uparrow}c_{\uparrow} -  c^{\dagger}_{\downarrow}c_{\downarrow} ) \end{equation}
\begin{equation} \tau_z \sigma_x = \sigma_z \otimes \sigma_x = \begin{pmatrix} \sigma_x & 0 \\ 0 & -\sigma_x \end{pmatrix}\to \alpha  c^{\dagger}_{\uparrow}c_{\downarrow} \mathrm{\ (spin-flip?)} \end{equation}
As defined in the paper, the 2D tight-binding Hamiltonian that is used is given by
$$H^{2D-TB} = \sum_{\mathbf{r}=x,y}\sum_{\sigma = \uparrow,\downarrow} -t (c^{\dagger}_{\mathbf{r+\hat{x}},\sigma} c^{}_{\mathbf{r},\sigma} + c^{\dagger}_{\mathbf{r+\hat{y}},\sigma} c^{}_{\mathbf{r},\sigma} ) + (4t-\mu)c^{\dagger}_{\mathbf{r},\sigma} c^{}_{\mathbf{r},\sigma} +\Delta_0 c^{\dagger}_{\mathbf{r},\uparrow} c^{\dagger}_{\mathbf{r},\downarrow} + \mathrm{h.c.} + J \sum_{\sigma,\sigma'}c^{\dagger}_{\mathbf{r},\sigma}(\mathbf{n(r) \cdot \vec{\sigma}})_{\sigma,\sigma'} c^{}_{\mathbf{r},\sigma'}  $$


In [3]:
#define a Boolean function to shape your system
r = 15
def disk(position): 
    x,y = position
    return x**2 + y**2 < r**2

def magn_texture(position,p):
    p = params.azi_winding
    q = params.radi_winding
    R = params.radius
    r = np.sqrt(x**2 + y**2)
#     if x is not 0:
#         t = np.arctan(y/x)
#     else:
#         t = np.arctan(np.inf*y)
    t = np.arctan(y/x) #be careful not to pass x=0!
    B = [np.sin(2*np.pi*q*(r/R))*np.cos(2*np.pi*q*t), np.sin(2*np.pi*q*(r/R))*np.sin(q*t), np.cos(2*np.pi*q*(r/R))]
    return B

def onsite(position,p): #define a function to determine the onsite energy term of the Hamiltonian
    B = magn_texture(position,p) #calculate direction of magnetic field at position (x,y)
    return 4*p['t'] - p['mu'] + p['j']*(B[0]*pauli.s0sx + B[1]*pauli.s0sy + B[2]*pauli.s0sz) + pauli.szs0*p['mu'] 
    
def hopping(x,y, p): #define the hopping terms in your system
    return -p['t']

In [108]:
params = SimpleNamespace(t=1, mu=-0.1, j=0.02, delta=0.2, B_0=0.1, azi_winding=2, radi_winding=1,radius=15)

#This was trying it with SimpleNamespace
def disk(position,params): 
    r = params.radius
    x,y = position
    return x**2 + y**2 < r**2

def magn_texture(position,params):
    x,y = position
    t = np.arctan2(y,x)
    p = params.aziwinding
    q = params.radiwinding
    R = params.radius
    r = np.sqrt(x**2 + y**2)
    B = [np.sin(2*np.pi*p*(r/R))*np.cos(q*t), np.sin(2*np.pi*p*(r/R))*np.sin(q*t), np.cos(2*np.pi*q*(r/R))]
    return B

def onsite(position,p): #define a function to determine the onsite energy term of the Hamiltonian
    x,y = position
    B = magn_texture((x,y),p)#calculate direction of magnetic field at position (x,y)
    return 4*p.t - p.mu + p.j*(B[0]*pauli.s0sx + B[1]*pauli.s0sy + B[2]*pauli.s0sz) + pauli.szs0*p.mu 
    
def hopping(x,y, p): #define the hopping terms in your system
    return -p.t


In [109]:
params = SimpleNamespace(t=1, mu=-0.1, j=0.02, delta=0.2, B_0=0.1, aziwinding=1, radiwinding=2,radius=15)
init = []
x = np.arange(-params.radius,params.radius,1)
y = np.arange(-params.radius,params.radius,1)
for i in x:
    for j in y:
        init.append(magn_texture((i,j),params))
C = np.reshape(np.asarray(init),(len(x),len(y),3));

In [114]:
hv.HoloMap({i: hv.Image(C[:,:,i], bounds=(x[0],y[0],-x[0],-y[0])).opts(colorbar=True, cmap='inferno') for i in range(3)}, kdims=('Direction'))

:HoloMap   [Direction]
   :Image   [x,y]   (z)

In [107]:
angle= [np.arctan2(,i)/np.pi for i in x]
angle

[0.9788106953650536,
 0.9773021290001396,
 0.9755627480278026,
 0.9735353239404101,
 0.9711420616236955,
 0.9682744825694465,
 0.9647767125227228,
 0.9604165758394345,
 0.9548327646991335,
 0.9474315432887466,
 0.937167041810999,
 0.9220208696226306,
 0.8975836176504333,
 0.8524163823495667,
 0.75,
 0.5,
 0.25,
 0.14758361765043326,
 0.10241638234956672,
 0.07797913037736932,
 0.06283295818900118,
 0.05256845671125343,
 0.045167235300866554,
 0.039583424160565546,
 0.03522328747727728,
 0.03172551743055357,
 0.028857938376304477,
 0.026464676059589878,
 0.024437251972197403,
 0.022697870999860426]

In [13]:
# Build your system, but don't calculate any matrices yet until you're happy with the result, which you can check with kwant.plot(sys).
sys = kwant.Builder() #initialize your system
sqlat = kwant.lattice.square()
sys[sqlat.shape(disk,(0,0))] = onsite #the tuple (0,0) sets the centre of the disk at the origin
#hopping in both directions is equal
sys[sqlat.neighbors()]= -1

# disk = kwant.plot(sys)

TypeError: disk() missing 1 required positional argument: 'params'

In [5]:
# import types
# class SimpleNamespace(types.SimpleNamespace):
#     def update(self, **kwargs):
#         self.__dict__.update(kwargs)
#         return self
#     def __getitem__(self, name):
#         return self.__dict__[name]
        
p = SimpleNamespace(t=1, mu=-0.1, j=0.02, delta=0.2, B0=0.1) #BUILDER GIVES ERROR WHEN TRYING TO PASS SIMPLENAMESPACE AS PARAMS
# p = dict(t=1,mu=0.1,j=0.02,delta=0.2,B0=0.1) #REGULAR DICT ALSO DIDNT WORK
# #define your parameters as tuples so that we can pass them through args

p.mu

-0.1

In [6]:
#Finalize the system when you're happy with the structure so that you can solve it and calculate the spectrum
ok = sys.finalized()

In [7]:
ham_np = ok.hamiltonian_submatrix(args=[p]) #returns explicit Hamiltonian as numpy array 
# eVal_np,eVec_np = np.linalg.eigh(ham_np)

# For very big systems it is more efficient to calculate the spectrum using sparse matrices
# ham =  ok.hamiltonian_submatrix(sparse=True)
# eVal_sp = la.eigsh(ham,k=ham.shape[0]-2)

C:\ProgramData\Anaconda3\envs\lps\lib\site-packages\ipykernel_launcher.py:1: KwantDeprecationWarning: The 'args' parameter has been deprecated since version 1.4 -- Instead, provide named parameters as a dictionary via 'params'.
  """Entry point for launching an IPython kernel.


UserCodeError: Error occurred in user-supplied value function "onsite".
See the upper part of the above backtrace for more information.

In [40]:
# plot energies
hv.Path((np.arange(len(eVal_np)),eVal_np))

:Path   [x,y]

In [8]:
disk.savefig('./fig/disk_hexlat_r={:d}.svg'.format(r))

In [12]:
np.arctan(np.pi/2)

1.0038848218538872